In [2]:
import sys
import os
import json
import xml.etree.ElementTree as ET
import glob

In [7]:
xml_dir = './annotations/' #Directory path to xml files.
img_dir = './setupdata/'
json_file = './setupdata/bbox_annotations.json' #Output COCO format json file."

In [8]:
#create a dictionary of filenames to relative file paths from img_dir
import os
filestruct = {}
for mapping in [(file,path) for path,dirs,files in os.walk(img_dir) for file in files]:
    filestruct[mapping[0]]=mapping[1].replace(img_dir,'').replace('\\','/')
filestruct

{'bbox_annotations.json': '',
 'ConvertVOCtoCOCO-checkpoint.ipynb': '.ipynb_checkpoints',
 '487akb3qhkks2qhhuc1pw9sb-wpengine_netdna-ssl_com_wp-content_uploads_2018_04_cracked-drywall-web-300x225.jpg': 'train_val_images/original_defects/crack',
 'allgaragefloors_com_wp-content_uploads_2017_05_concrete-crack-repair-1.jpg': 'train_val_images/original_defects/crack',
 'as2_ftcdn_net_jpg_01_82_30_17_500_F_182301746_sKGU4TxY338sHT9v0iYYEbu4Yu0FYAHG.jpg': 'train_val_images/original_defects/crack',
 'blog_master-builders-solutions_basf_com_hs-fs_hubfs_BASFBlog_imgs_concrete-crack-bodywidth300nameconcrete-crack-body.jpg': 'train_val_images/original_defects/crack',
 'blog_master-builders-solutions_basf_com_hubfs_BASFBlog_imgs_concrete-crack-body.jpg': 'train_val_images/original_defects/crack',
 'bloximages_chicago2_vip_townnews_com_bozemandailychronicle_com_content_tncms_assets_v3_editorial_3_64_3644817a-4284-11df-a246-001cc4c002e0_3644817a-4284-11df-a246-001cc4c002e0_image.jpg': 'train_val_ima

In [9]:
START_BOUNDING_BOX_ID = 1
PRE_DEFINE_CATEGORIES = None
# If necessary, pre-define category and its id
#  PRE_DEFINE_CATEGORIES = {"aeroplane": 1, "bicycle": 2, "bird": 3, "boat": 4,
#  "bottle":5, "bus": 6, "car": 7, "cat": 8, "chair": 9,
#  "cow": 10, "diningtable": 11, "dog": 12, "horse": 13,
#  "motorbike": 14, "person": 15, "pottedplant": 16,
#  "sheep": 17, "sofa": 18, "train": 19, "tvmonitor": 20}


def get(root, name):
    vars = root.findall(name)
    return vars


def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise ValueError("Can not find %s in %s." % (name, root.tag))
    if length > 0 and len(vars) != length:
        raise ValueError(
            "The size of %s is supposed to be %d, but is %d."
            % (name, length, len(vars))
        )
    if length == 1:
        vars = vars[0]
    return vars


def get_categories(xml_files):
    """Generate category name to id mapping from a list of xml files.
    
    Arguments:
        xml_files {list} -- A list of xml file paths.
    
    Returns:
        dict -- category name to id mapping.
    """
    classes_names = []
    for xml_file in xml_files:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall("object"):
            classes_names.append(member[0].text)
    classes_names = list(set(classes_names))
    classes_names.sort()
    return {name: i for i, name in enumerate(classes_names)}


def convert(xml_files, json_file):
    json_dict = {"images": [], "type": "instances", "annotations": [], "categories": []}
    if PRE_DEFINE_CATEGORIES is not None:
        categories = PRE_DEFINE_CATEGORIES
    else:
        categories = get_categories(xml_files)
    bnd_id = START_BOUNDING_BOX_ID
    for idx, xml_file in enumerate(xml_files):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        path = get(root, "path")
        filename = get(root, "filename")
        
        filename = os.path.basename(filename[0].text)
        print(filename)
        directory = None
        directory = filestruct[filename]
        image_id = idx#get_filename_as_int(filename)
        size = get_and_check(root, "size", 1)
        width = int(get_and_check(size, "width", 1).text)
        height = int(get_and_check(size, "height", 1).text)
        image = {
            "file_name": os.path.join('./',directory,filename),
            "height": height,
            "width": width,
            "id": image_id,
        }
        json_dict["images"].append(image)
        ## Currently we do not support segmentation.
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, "object"):
            category = get_and_check(obj, "name", 1).text
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, "bndbox", 1)
            xmin = int(get_and_check(bndbox, "xmin", 1).text) - 1
            ymin = int(get_and_check(bndbox, "ymin", 1).text) - 1
            xmax = int(get_and_check(bndbox, "xmax", 1).text)
            ymax = int(get_and_check(bndbox, "ymax", 1).text)
            assert xmax > xmin
            assert ymax > ymin
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {
                "area": o_width * o_height,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [xmin, ymin, o_width, o_height],
                "category_id": category_id,
                "id": bnd_id,
                "ignore": 0,
                "segmentation": [],
            }
            json_dict["annotations"].append(ann)
            bnd_id = bnd_id + 1

    for cate, cid in categories.items():
        cat = {"supercategory": "none", "id": cid, "name": cate}
        json_dict["categories"].append(cat)

    os.makedirs(os.path.dirname(json_file), exist_ok=True)
    json_fp = open(json_file, "w")
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()



In [10]:
xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))

# If you want to do train/test split, you can pass a subset of xml files to convert function.
print("Number of xml files: {}".format(len(xml_files)))
convert(xml_files, json_file)
print("Success: {}".format(json_file))

Number of xml files: 103
20190925_152629.jpg
20190925_152634.jpg
20190925_152638.jpg
20190925_152643.jpg
20190925_152646.jpg
20190925_152653.jpg
20190925_152733.jpg
20190925_152752.jpg
20190925_152800.jpg
20190925_152814.jpg
20190925_152819.jpg
20190925_152829.jpg
20190925_152836.jpg
20190925_152840.jpg
20190925_152847.jpg
20190925_152849.jpg
20190925_152905.jpg
20190925_152938.jpg
20190925_153027.jpg
20190925_153037.jpg
20190925_153043.jpg
20190925_153047.jpg
20190925_153100.jpg
20190925_153120.jpg
20190925_153229.jpg
20190925_153236.jpg
20190925_153242.jpg
20190925_153246.jpg
20190925_153254.jpg
20190925_153333.jpg
20190925_153345.jpg
20190925_153348.jpg
IMG_2350.JPG
IMG_2351.JPG
IMG_2352.JPG
IMG_2355.JPG
IMG_2356.JPG
IMG_2357.JPG
IMG_2358.JPG
IMG_2359.JPG
IMG_2361.JPG
IMG_2364.JPG
IMG_2365.JPG
IMG_2366.JPG
IMG_2367.JPG
IMG_2368.JPG
IMG_2375.JPG
IMG_2376.JPG
IMG_2377.JPG
IMG_2380.JPG
IMG_2381.JPG
IMG_2383.JPG
IMG_2387.JPG
IMG_2388.JPG
IMG_2390.JPG
IMG_2392.JPG
IMG_2393.JPG
IMG_2402.J